In [1]:
import logging
import grpc
import pandas as pd
import numpy as np
import time
import sys
sys.path.append('proto')
from proto import common_pb2,common_pb2_grpc
from proto import contest_pb2,contest_pb2_grpc,question_pb2,question_pb2_grpc
import talib



In [2]:
df = pd.read_csv("./data/CONTEST_DATA_IN_SAMPLE_1.csv",header=None)
df.columns = ['time','stkcd','open','high','low','close','volume']

In [14]:
%%time
start = time.time()
df.groupby('stkcd')['open'].rolling(10).max()
end = time.time()
print(end-start)

0.21841716766357422
Wall time: 218 ms


In [4]:
def get_factors(data):
    '''
    @params:
    data: 2D-array, rows as daily numbers, columns as open,high,low,close,volume respectively

    @returns:
    factors: 1D-array
    '''
    o = data[:,0]
    h = data[:,1]
    l = data[:,2]
    c = data[:,3]
    v = data[:,4]
    mom = talib.MOM(c,timeperiod=10)
    vol = talib.STDDEV(c,timeperiod=10)
    max52 = talib.MAX(h,timeperiod=252)
    min52 = talib.MIN(l,timeperiod=252)

    # 价量相关性、下行波动占比，ij是什么？
    # 
    # clr =  # close_volume relation
    # dvr =  # downstream volatility ratio

    # Commodity Channel Index
    cci = talib.CCI(h,l,c,timeperiod=14)

    # KDJ (Stochastic)
    K, D = talib.STOCH(h,l,c,fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    J = 3 * D - 2 * K

    # Relative Strength Index
    rsi = talib.RSI(c,timeperiod=14)

    # TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
    trix = talib.TRIX(c, timeperiod=30)

    # Williams' %R
    willr = talib.WILLR(h, l, c, timeperiod=14)

    # Moving Average Convergence/Divergence
    macd, macdsignal, macdhist = talib.MACD(c, fastperiod=12, slowperiod=26, signalperiod=9)

    # Normalized Average True Range
    atr = talib.NATR(h,l,c,timeperiod=14)

    return np.concatenate([data,np.asarray([mom, vol, max52, min52, cci, K, D, J, rsi, trix, willr, macd, macdsignal, macdhist, atr]).transpose()],axis=1)


In [10]:
%time
# from utils import get_factors
b = get_factors(df[['open','high','low','close','volume']].values)

Wall time: 0 ns


In [13]:
%time
start = time.time()
df2 = df.groupby('stkcd').apply(lambda x: get_factors(x[['open','high','low','close','volume']].values))
end = time.time()
print(end-start)

Wall time: 0 ns
0.31618571281433105


In [ ]:
df.groupby('stkcd')['']

array([[19.96632   , 19.96632   , 19.06278   , ...,         nan,
                nan,         nan],
       [19.86138   , 20.59662   , 19.06278   , ...,         nan,
                nan,         nan],
       [20.9748    , 21.64734   , 20.82762   , ...,         nan,
                nan,         nan],
       ...,
       [17.30982   , 17.30982   , 16.83462   , ..., -0.19497894,
        -0.11294295,  2.42823031],
       [17.23788   , 17.76786   , 16.48152   , ..., -0.23186479,
        -0.14754336,  2.86057332],
       [16.46502   , 16.46502   , 15.76674   , ..., -0.28311416,
        -0.20499749,  3.10048582]])

In [28]:
%%time
talib.MAX(df.groupby('stkcd')['open'],10)
# talib.STDDEV()
# df['open'].rolling(10).max()

TypeError: Argument 'real' has incorrect type (expected numpy.ndarray, got SeriesGroupBy)

In [10]:
contest_channel=grpc.insecure_channel('47.103.23.116: 56702')
question_channel=grpc.insecure_channel('47.103.23.116: 56701')

contest_stub=contest_pb2_grpc.ContestStub(contest_channel)
question_stub=question_pb2_grpc.QuestionStub(question_channel)

login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

In [15]:
sequence = 0
last_sequence = 0
while sequence < 1000:
    question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88))
    sequence = question_response.sequence
    if last_sequence < sequence:
        # Run our strategy here, resulting in a POS array with length 351

        dailystk = [x.values for x in question_response.dailystk]
        ds = np.asarray(dailystk)

        # Here is our target array
        # pos = np.sign(np.random.rand(351) - 0.5) * np.floor(question_response.capital / 351 / ds[:,5])
        # pos = pos * -1
        # summit answer
        # submit_response = contest_stub.submit_answer(contest_pb2.AnswerRequest(user_id=88,user_pin='dDTSvdwk',session_key=login_response.session_key,sequence=sequence,positions=pos))
        print(submit_response)
        time.sleep(1)
        break

accepted: true



In [16]:
question_response.positions

[58580.0, 43253.0, -15725.0, -143889.0, 0.0, 0.0, 6389.0, 52642.0, 0.0, -7694.0, -8879.0, 17633.0, -3609.0, 0.0, -21912.0, 0.0, 11217.0, 0.0, -5520.0, 134895.0, -25392.0, 0.0, -17349.0, 37998.0, -71705.0, -1901.0, -25694.0, -21801.0, 0.0, -3681.0, 83000.0, 20555.0, -5225.0, 4075.0, 0.0, -20539.0, 0.0, -5242.0, -47963.0, 3246.0, 10818.0, 10285.0, 11463.0, 0.0, -9164.0, 6991.0, 8622.0, -8773.0, 33514.0, 32951.0, 0.0, -83656.0, 0.0, -138817.0, 5632.0, -5394.0, -13306.0, 208817.0, 113411.0, 2365.0, -70997.0, 0.0, -75731.0, 0.0, 15527.0, 43419.0, 123347.0, -3002.0, 41111.0, 13335.0, 36644.0, 0.0, 0.0, -5633.0, 0.0, -2822.0, -12607.0, 0.0, -4594.0, -9735.0, 42570.0, 0.0, 0.0, 0.0, 19801.0, -215833.0, 0.0, -23051.0, 13574.0, -50784.0, 6435.0, 104000.0, -20096.0, -45030.0, 0.0, -4803.0, 13560.0, 0.0, 0.0, 0.0, -7805.0, -6552.0, 0.0, 39457.0, 40723.0, 5441.0, -12809.0, -41111.0, 189327.0, -31159.0, 0.0, 0.0, -31416.0, -8965.0, 0.0, -19603.0, 22892.0, 25593.0, -126218.0, 5374.0, -110683.0, -9946

In [17]:
pos

array([  58580.,   43253.,  -15725., -143889.,    6702.,  -21077.,
          6389.,   52642.,  -37744.,   -7694.,   -8879.,   17633.,
         -3609.,   -6254.,  -21912., -263147.,   11217.,   40043.,
         -5520.,  134895.,  -25392.,   22912.,  -17349.,   37998.,
        -71705.,   -1901.,  -25694.,  -21801.,  146327.,   -3681.,
         83000.,   20555.,   -5225.,    4075.,  -34204.,  -20539.,
         -8809.,   -5242.,  -47963.,    3246.,   10818.,   10285.,
         11463.,  -22647.,   -9164.,    6991.,    8622.,   -8773.,
         33514.,   32951.,   37406.,  -83656.,   26035., -138817.,
          5632.,   -5394.,  -13306.,  208817.,  113411.,    2365.,
        -70997.,   19964.,  -75731.,  -55059.,   15527.,   43419.,
        123347.,   -3002.,   41111.,   13335.,   36644.,   -8970.,
         98106.,   -5633.,   17850.,   -2822.,  -12607.,    9176.,
         -4594.,   -9735.,   42570.,   14060.,   15943.,   32702.,
         19801., -215833.,  -49869.,  -23051.,   13574.,  -507

In [13]:
submit_response.accepted

True

In [4]:
question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88))

In [5]:
question_response.capital

483828965.01277775